In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv
load_dotenv(override=True)
import google.generativeai as genai
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings

c:\Users\aryam\Documents\ML\Ada_E_Comm\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Pre-Work

In [51]:
os.getcwd()

'c:\\Users\\aryam\\Documents\\ML\\Ada_E_Comm\\Ada'

In [54]:
prin

NameError: name 'prin' is not defined

In [ ]:
# df = pd.read_json('Final_Dataset_JSON.json',encoding='utf-8',orient='records')  comment to keep it safe from running again

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6629 entries, 0 to 6628
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Main Category    6629 non-null   object 
 1   Tag              6629 non-null   object 
 2   ASIN             6629 non-null   object 
 3   Product Link     6629 non-null   object 
 4   Images           6629 non-null   object 
 5   title            6629 non-null   object 
 6   productOverview  5674 non-null   object 
 7   featureBullets   5878 non-null   object 
 8   stars            6356 non-null   float64
 9   ratings          6356 non-null   float64
 10  listPrice        6288 non-null   float64
 11  salePrice        6629 non-null   int64  
 12  bookDescription  647 non-null    object 
dtypes: float64(3), int64(1), object(9)
memory usage: 673.4+ KB


In [ ]:
df.columns

Index(['Main Category', 'Tag', 'ASIN', 'Product Link', 'Images', 'title',
       'productOverview', 'featureBullets', 'stars', 'ratings', 'listPrice',
       'salePrice', 'bookDescription'],
      dtype='object')

In [ ]:
df.isnull().sum()

Main Category         0
Tag                   0
ASIN                  0
Product Link          0
Images                0
title                 0
productOverview     955
featureBullets      751
stars               273
ratings             273
listPrice           341
salePrice             0
bookDescription    5982
dtype: int64

In [ ]:
(df[~df['productOverview'].isna()]['productOverview'].apply(lambda x : True if isinstance(x,dict) else False)).all()

np.True_

In [ ]:
(df[~df['featureBullets'].isna()]['featureBullets'].apply(lambda x : True if isinstance(x,list) else False)).all()

np.True_

In [ ]:
(df[~df['bookDescription'].isna()]['Main Category'] == 'Books').all()

np.True_

In [ ]:
len(df['ASIN'].unique())

6629

In [ ]:
# texts = []  comment to keep it safe

In [ ]:

# I have commented the code below to make it un-runable

# for _, row in df.iterrows():
#     main_cat = row['Main Category']
#     tag = row['Tag']
#     title = row['title']
#     asin = row['ASIN']
#     overview = row['productOverview']
#     features = row['featureBullets']
#     stars = row['stars']
#     ratings = row['ratings']
#     listprice = row['listPrice']
#     saleprice = row['salePrice']
#     bookdescription = row['bookDescription']

#     parts = []
#     parts.append(f"Top Category of the product : {main_cat}")
#     parts.append(f"Sub-Category of the product : {tag}")
#     parts.append(f"Title of the product : {title}")
#     parts.append(f"ASIN Id of the product : {asin}")
    
#     if overview:
#         for j in overview:
#             parts.append(f"{j} : {overview[j]}")

#     if features:
#         parts.append(f"Features : {' '.join(features)}")

#     if stars:
#         parts.append(f"Stars given by Users : {stars}")

#     if ratings:
#         parts.append(f"Ratings given by Users : {ratings}")

#     if listprice:
#         parts.append(f"List Price of the product : {listprice}")

#     parts.append(f"Sale Price of the product : {saleprice}")

#     if bookdescription:
#         parts.append(f"Description of the Book : {bookdescription}")

#     text = "\n".join(parts)

#     texts.append(text)


    


In [55]:
len(texts)

6629

# Generating Vector Embeddings

In [ ]:
import os
from dotenv import load_dotenv
import streamlit as st
import google.generativeai as genai


from config import process_pdf


load_dotenv(override=True)



def user_input(user_question):
        embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
        
        new_db = FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)
        docs = new_db.similarity_search(user_question)

        chain = process_pdf.get_conversational_chain()

        
        response = chain(
            {"input_documents":docs, "question": user_question}
            , return_only_outputs=True)

        print(response)
        st.write("Reply: ", response["output_text"])


def main():
    st.set_page_config("PDF Q&A Chatbot", page_icon = ":scroll:")
    st.header("PDF📚 - Chat BOt 🤖 ")

    user_question = st.text_input("Ask Question from the PDF Files uploaded .. ✍️📝")

    if user_question:
        user_input(user_question)

    with st.sidebar:

        st.image("logo.jpg")
        st.write("---")
        
        st.title("📁 PDF File's Section")
        pdf_docs = st.file_uploader("Upload your PDF Files & \n Click on the Submit Button ", accept_multiple_files=True)
        if st.button("Submit"):
            with st.spinner("Processing..."): # user friendly message.
                raw_text = process_pdf.get_pdf_text(pdf_docs) # get the pdf text
                text_chunks = process_pdf.get_text_chunks(raw_text) # get the text chunks
                process_pdf.get_vector_store(text_chunks) # create vector store
                st.success("Done")
        

if __name__ == "__main__":
    main()


In [58]:
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
vector_store = FAISS.from_texts(texts, embedding=embeddings)
vector_store.save_local("faiss_index")